In [55]:
import pandas as pd
import numpy as np
import sqlite3
import importlib

# keeping company information in additional file
import data_file

In [15]:
sales_person_n = data_file.sales_person_n
exlbu=data_file.exlbu
exlpanrter=data_file.exlpanrter

In [95]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"  # Replace 'tablename' with your table name
df_orders = pd.read_sql_query(query, conn)
conn.close()

conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  # Replace 'tablename' with your table name
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [96]:
df_orders['sold_to_customer'] = df_orders['sold_to_customer'].astype(str)
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
df_orders[~df_orders['sold_to_customer'].isin(df_customers['sold_to_customer'].to_list())]

,company_code_n,year_month,Date,FY,customer_group_code,source,bu2,bu_n,sales_person_n,sales_order_so,...,sold_to_customer_n,sold_to_region_n,eu_industry_n,ec_eu_customer_n,eu_region_n,ec_eu_industry_n,order_intake_amount_eur,sold_to_country_n,ec_eu_customer,ec_eu_country_n


In [98]:
len(df_customers)

14657

In [40]:
# for testing purposes I keep this lines
wdf = df_customers.copy()
dfc = df_orders.copy()

In [41]:
len(dfc)

202769

In [42]:
bu_defin = pd.read_excel('data_files/bu_defin.xlsx')
bu_defin = bu_defin.loc[:, 'bu2':]
bu_defin['for_bu'] = bu_defin['for_bu'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].str.strip()

In [43]:
order_data = dfc.loc[:,['company_code_n', 'year_month', 'FY', 
       'bu2', 'sales_person_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n','order_intake_amount_eur']]

order_data['bu2'] = order_data['bu2'].astype(str)
order_data['bu2'] = order_data['bu2'].str.strip()

# add bu column for future reports
order_data = order_data.merge(bu_defin, how='left' )

In [21]:
len(order_data)

202769

In [44]:
order_columns = list(order_data.columns) + ['customer_name', 'indirect_direct', 'tier', 'type', exlpanrter]

In [45]:
# special case with specific SO for agent
agent_so = data_file.special_so
agent_so_two = data_file.special_so_two 
agent_one = data_file.agent_one
so_order_data = order_data[(order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two)]
print(len(so_order_data))
order_data = order_data[~((order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two))]
print(len(order_data))

so_order_data['customer_name'] = agent_one
so_order_data['indirect_direct'] = 'Indirect'
so_order_data['tier'] = 'Channel Partner'
so_order_data['type'] = 'Agent'

2
202767


In [46]:
# special case with specific cutomers for agent
partners_data = wdf.copy()
agent_two = data_file.agent_two
partners_data.loc[partners_data[exlpanrter].notna(), 'customer_name'] = agent_two
partners_data = partners_data.loc[:, ['sold_to_customer', 'agent_person', 'company_code_n', 'sold_to_customer_n','customer_name', 
       'indirect_direct', 'channel', 'type', 'tier', exlpanrter]]

partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)

In [47]:
agent_partners = partners_data[partners_data['agent_person'].notna()]
# agent sales records preparation
agents_order_data = order_data[order_data['sales_person_n'].isin(sales_person_n)]
print(len(order_data))
print(len(agents_order_data))
order_data = order_data[~order_data['sales_person_n'].isin(sales_person_n)]
print(len(order_data))
agents_order_data = agents_order_data.merge(agent_partners, left_on='sales_person_n', right_on='sold_to_customer_n', how='left')
agents_order_data = agents_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
agents_order_data.columns = agents_order_data.columns.str.replace('_x', '')
agents_order_data = agents_order_data.loc[:, order_columns]
agents_order_data['type'] = 'Agent'

202767
3247
199520


In [48]:
# all partners except agents data preparation
"""other_order_data = order_data[order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
print(len(order_data))
print(len(other_order_data))
checkthis = order_data[~order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
print(len(checkthis))"""

other_order_data = order_data.copy()

other_order_data = other_order_data.merge(partners_data, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
other_order_data = other_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
other_order_data.columns = other_order_data.columns.str.replace('_x', '')
other_order_data = other_order_data.loc[:, order_columns]
# Update values in columns based on the specific value
other_order_data.loc[((other_order_data['indirect_direct'] == 'Direct') & ( (other_order_data['tier'].notna()) | (other_order_data[exlpanrter].notna()) )), 'tier'] = 'Channel Partner'
other_order_data.loc[((other_order_data['indirect_direct'] == 'Direct') & ( (other_order_data['tier'].notna()) | (other_order_data[exlpanrter].notna()) )), 'type'] = 'Agent'                                                                       

In [ ]:
checkthis['sold_to_customer_n'].unique()

In [49]:
len(other_order_data)

199520

In [38]:
199445 + 3247 + 2 - 202767

-73

In [50]:
# combining all the sales records
full_data = pd.concat([other_order_data, agents_order_data, so_order_data])
full_data.reset_index(inplace=True, drop=True)
#full_data.drop_duplicates(inplace=True)

In [51]:
len(full_data)

202769

In [52]:
writer = pd.ExcelWriter('data_files/outcome/full_data.xlsx')
# Save each DataFrame to a separate sheet in the same file
full_data.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()

In [86]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"  # Replace 'tablename' with your table name
df_orders = pd.read_sql_query(query, conn)
conn.close()

In [76]:
conn = sqlite3.connect('data_files/order_data.db')
query = "DELETE FROM orders WHERE year_month = 202305"
conn.execute(query)
conn.commit()
conn.close()

In [75]:
len(df_orders) - len(df_orders[df_orders['year_month'] == 202305])

201027

In [87]:
len(df_orders)

200981

In [83]:
201027 - 38

200989

In [88]:
df_orders['sold_to_customer_n'] = df_orders['sold_to_customer_n'].fillna('')
df_orders[df_orders['sold_to_customer_n'].str.contains('Yokogawa')] 

,company_code_n,year_month,Date,FY,customer_group_code,source,bu2,bu_n,sales_person_n,sales_order_so,...,sold_to_customer_n,sold_to_region_n,eu_industry_n,ec_eu_customer_n,eu_region_n,ec_eu_industry_n,order_intake_amount_eur,sold_to_country_n,ec_eu_customer,ec_eu_country_n


In [85]:
conn = sqlite3.connect('data_files/order_data.db')
query = "DELETE FROM orders WHERE sold_to_customer_n LIKE '%Yokogawa%'"
conn.execute(query)
conn.commit()
conn.close()

In [90]:
df_orders['sold_to_customer'] = df_orders['sold_to_customer'].astype(str)
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
xxx = df_orders[~df_orders['sold_to_customer'].isin(df_customers['sold_to_customer'].to_list())]

In [93]:
xxx = xxx.loc[:, ['company_code_n', 'customer_group_code', 'sold_to_customer', 'sold_to_customer_n']]

In [94]:
writer = pd.ExcelWriter('data_files/outcome/xxx.xlsx')
# Save each DataFrame to a separate sheet in the same file
xxx.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()